`data/englishonset` 데이터를 학습합시다.

In [1]:
from phoncon.corpus import LearningData, slice_word, randomize_word
from phoncon.features import FeatureSystem

LEARNING_DATA = "./data/englishonset/EnglishLearningData.txt"
FEATURES = "./data/englishonset/EnglishFeatures.txt"

ld = LearningData(LEARNING_DATA, prefix="# ")
fs = FeatureSystem.load(FEATURES) \
    .add_word_boundary("#", "word_boundary") \
    .to_binary_feature()

In [32]:
N = 1000
well_formed = ld.sample(N)
ill_formed = randomize_word(ld.sample(N), fs.get_phonemes())

In [33]:
wf_sliced = list(slice_word(well_formed, 2))
if_sliced = list(slice_word(ill_formed, 2))

X = fs.get_matrix(wf_sliced + if_sliced)
y = len(wf_sliced) * [1] + len(if_sliced) * [0]

In [40]:
from sklearn.tree import plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

dtc = RandomForestClassifier(
    n_estimators=100,
    max_depth=5,
)
dtc.fit(X, y)
y_pred = dtc.predict(X)

In [41]:
accuracy_score(y, y_pred)

0.741504854368932

In [43]:
import matplotlib.pyplot as plt

plt.figure(figsize=(24, 16))

plot_tree(dtc.estimators_[0], feature_names=X.columns, filled=True)
plt.savefig("decision_tree.png")
plt.close()

In [48]:
indice = X.query("`-word_boundary0` == 1 & `+voice1` == 1").index
indice
# [y[i] for i in indice]

Index(['T DH', 'NG G', 'G B', 'S B', 'DH ZH', 'Z B', 'Y JH', 'CH D', 'D V',
       'G G', 'P G', 'D V', 'M G', 'SH V', 'Y ZH', 'DH V', 'D JH', 'NG V',
       'S V', 'D B', 'S B', 'S DH', 'G Z', 'Y Z', 'ZH D', 'D ZH', 'D B',
       'CH V', 'V Z', 'T B', 'SH ZH', 'Z G', 'HH DH', 'V D', 'L D', 'K ZH',
       'W Z', 'G JH', 'D Z', 'S G', 'JH G', 'F D', 'T JH', 'Y B', 'F ZH',
       'NG V', 'V JH', 'V JH', 'T Z', 'S DH', 'ZH JH', 'M JH', 'DH D', 'NG JH',
       'S Z', 'HH JH', 'P DH', 'B B', 'M B', 'L Z', 'G Z', 'R B', 'V ZH',
       'G Z', 'W Z', 'JH DH', 'CH B'],
      dtype='object')

In [49]:
fs._data.query("`+voice` == 1")

,+consonantal,-consonantal,+approximant,-approximant,+sonorant,-sonorant,+continuant,-continuant,+nasal,-nasal,...,+lateral,-lateral,+dorsal,-dorsal,+high,-high,+back,-back,+word_boundary,-word_boundary
B,1,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
D,1,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
DH,1,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
G,1,0,0,1,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,1
JH,1,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
V,1,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
Z,1,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
ZH,1,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [50]:
# -continuant1 == 0, -approximant1 == 0, -voice0 == 1

In [5]:
# 데이터 만든다

# 적당히 변환한다 (X, y)
# Decision Tree 돌린다
# 결과를 Visualize 한다.